In [ ]:
import requests
import base64


In [ ]:
# replace with your own client id and client secret
CLIENT_ID = '745ed33b9c3e4aa2b2d0dc485c0811bf'
CLIENT_SECRET = '8e098794b05748a3bd69af74170d6f0e'

# Base64 encode the client id and client secret
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

In [ ]:
# request the access token
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url, data=data, headers=headers)
if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained successfully.")
else:
    print("Error obtaining access token.")
    exit()

Access token obtained successfully.


In [ ]:
!pip install spotipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.3/261.3 kB 6.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth


In [ ]:
def get_trending_playlist_data(playlist_id, access_token):
    # set up spotipy with the access token
    sp = spotipy.Spotify(auth=access_token)

    # get the tracks from the playlist
    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))')

    # extract relevant information and store in a list of dictionaries
    music_data = []
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']

        # get audio features for the track
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None

        # get release date of the album
        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None

        # get popularity of the track
        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None

        # add additional track information to the track data
        track_data = {
            'Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None),
            'External URLs': track_info.get('external_urls', {}).get('spotify', None),
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,
            # add more attributes as needed (go through the documentation to know what more you can add)
        }

        music_data.append(track_data)

    # create a pandas dataframe from the list of dictionaries
    df = pd.DataFrame(music_data)

    return df

In [ ]:
# you can add the playlist id of any playlist on Spotify here
playlist_id = '1YQzSpN1WqojjGtVgZK41E'

# call the function to get the music data from the playlist and store it in a DataFrame
music_df = get_trending_playlist_data(playlist_id, access_token)

print(music_df)

                 Track Name                          Artists  \
0          Nazende sevgilim  Andrey Babayev, Rashid Beibutov   
1   Kuchalara Soo Sapmisham                  Rashid Beibutov   
2                  Sevgilim                  Rashid Behbudov   
3     Bir Könül Sındırmışam                Şövkət Ələkbərova   
4   Girdim Yarim Baghchsina                  Rashid Beibutov   
..                      ...                              ...   
95     Səndə Qalıbdır Gözüm                  Ağadadaş Ağayev   
96   Yar Bizə Qonaq Gələcək                  Rashid Behbudov   
97            Dinə Bilmədim                 Mənsum İbrahimov   
98         Gedək Üzü Küləyə                Şövkət Ələkbərova   
99            Sənsən Ürəyim                Şövkət Ələkbərova   

                                      Album Name                Album ID  \
0                               Baku gözel çeher  39XizleGU3i6soI6CrKWfT   
1                               Arshin Mul Ullun  4kCVrZiWYSVAyb82SGkWft   
2  

In [ ]:
music_df.to_csv("musicdata.csv")

In [ ]:
music_df

,Track Name,Artists,Album Name,Album ID,Track ID,Popularity,Release Date,Duration (ms),Explicit,External URLs,...,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,Nazende sevgilim,"Andrey Babayev, Rashid Beibutov",Baku gözel çeher,39XizleGU3i6soI6CrKWfT,4FAHuifWNBufoQzqyoPhAX,37,2019-07-07,187107,False,https://open.spotify.com/track/4FAHuifWNBufoQz...,...,0.2590,6,-10.505,1,0.0417,0.971,0.000000,0.3860,0.330,83.455
1,Kuchalara Soo Sapmisham,Rashid Beibutov,Arshin Mul Ullun,4kCVrZiWYSVAyb82SGkWft,362m7XVWdOU7FsnliFZFHI,32,1995,182047,False,https://open.spotify.com/track/362m7XVWdOU7Fsn...,...,0.0461,4,-16.501,0,0.0431,0.996,0.131000,0.0752,0.257,70.901
2,Sevgilim,Rashid Behbudov,Tələsiin İnsanlar,17GM8oatGxfNSa5JSSPeQO,1cH9IYQUEsfvvw42kvWQBc,19,2001,178827,False,https://open.spotify.com/track/1cH9IYQUEsfvvw4...,...,0.2200,9,-14.562,0,0.0342,0.925,0.000000,0.1180,0.217,98.629
3,Bir Könül Sındırmışam,Şövkət Ələkbərova,Bir Könül Sındırmışam,2fXYCKtQ3ttwhAOns6kNvq,4XdWoVnlcIZuCchQYIVmyr,0,1987-01-24,322432,False,https://open.spotify.com/track/4XdWoVnlcIZuCch...,...,0.3020,4,-11.250,0,0.0352,0.982,0.008780,0.4260,0.457,111.934
4,Girdim Yarim Baghchsina,Rashid Beibutov,Arshin Mul Ullun,4kCVrZiWYSVAyb82SGkWft,5a1rhTh2pDrZTy5JeO4ubP,31,1995,200255,False,https://open.spotify.com/track/5a1rhTh2pDrZTy5...,...,0.1830,11,-10.420,0,0.0350,0.987,0.000687,0.1010,0.194,82.317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Səndə Qalıbdır Gözüm,Ağadadaş Ağayev,Səndə Qalıbdır Gözüm,58VIJVkvRlOJwNor09S6Bg,6OJOdwoFCWDOiZYQCUGKIN,18,2020-03-01,268934,False,https://open.spotify.com/track/6OJOdwoFCWDOiZY...,...,0.8300,11,-10.124,0,0.0508,0.755,0.000005,0.0888,0.778,129.705
96,Yar Bizə Qonaq Gələcək,Rashid Behbudov,Qalalıyam Qalalı (Azərbaycan Xalq Mahnıları),24rzZ4GRU1GcPrVsHEzrWa,3EyMnebawYAw9rdRM0xkpZ,18,2001,168467,False,https://open.spotify.com/track/3EyMnebawYAw9rd...,...,0.2730,4,-11.012,1,0.0425,0.993,0.000021,0.1540,0.704,175.822
97,Dinə Bilmədim,Mənsum İbrahimov,Dinə Bilmədim,4EuKX0pOH2gfMA1mAUpUsP,5oQbWLdIKQdLIPQ2cImcy4,20,2007-10-01,259265,False,https://open.spotify.com/track/5oQbWLdIKQdLIPQ...,...,0.4150,11,-8.414,0,0.0411,0.723,0.000000,0.2280,0.743,105.266
98,Gedək Üzü Küləyə,Şövkət Ələkbərova,"İfalar, Vol. 8",6YZ9sAxWgDrugPfXGw7Bj0,19GXitsfTqUENNXdXOMqJJ,16,2022-08-17,262881,False,https://open.spotify.com/track/19GXitsfTqUENNX...,...,0.1890,7,-13.361,0,0.0361,0.992,0.852000,0.1190,0.361,110.272


In [ ]:
music_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Track Name        100 non-null    object 
 1   Artists           100 non-null    object 
 2   Album Name        100 non-null    object 
 3   Album ID          100 non-null    object 
 4   Track ID          100 non-null    object 
 5   Popularity        100 non-null    int64  
 6   Release Date      100 non-null    object 
 7   Duration (ms)     100 non-null    int64  
 8   Explicit          100 non-null    bool   
 9   External URLs     100 non-null    object 
 10  Danceability      100 non-null    float64
 11  Energy            100 non-null    float64
 12  Key               100 non-null    int64  
 13  Loudness          100 non-null    float64
 14  Mode              100 non-null    int64  
 15  Speechiness       100 non-null    float64
 16  Acousticness      100 non-null    float64
 17

In [ ]:
music_df.describe()

,Popularity,Duration (ms),Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,17.090000,254765.870000,0.474910,0.450315,5.620000,-10.508150,0.310000,0.054735,0.727962,0.086628,0.214701,0.516319,114.716440
std,9.478412,75847.606926,0.140564,0.234972,3.637015,3.512201,0.464823,0.036086,0.280913,0.211794,0.120558,0.258680,32.059161
min,0.000000,96496.000000,0.160000,0.046100,0.000000,-18.618000,0.000000,0.028300,0.017600,0.000000,0.053400,0.083900,61.893000
25%,11.000000,200144.500000,0.380750,0.269250,2.750000,-12.970750,0.000000,0.035525,0.594500,0.000004,0.116750,0.284250,88.864500
50%,16.000000,246725.500000,0.469000,0.414500,5.000000,-10.651500,0.000000,0.042400,0.804000,0.000158,0.180000,0.509000,111.103000
75%,23.000000,299159.500000,0.588000,0.629250,9.000000,-7.924750,1.000000,0.056700,0.971250,0.010332,0.287250,0.738000,129.847750
max,38.000000,479936.000000,0.825000,0.981000,11.000000,-0.742000,1.000000,0.266000,0.996000,0.852000,0.621000,0.971000,195.243000



Here are some key insights from the summary statistics in both Azerbaijani and English:


Track Duration: The average track length is approximately 4.25 minutes, with the longest track being around 8 minutes.
Popularity: The average popularity score is 17.6, with a maximum value of 39 and a minimum of 0. Most tracks have a lower popularity score, as seen from the percentiles.
Energy Levels: The average energy score is 0.45, with a high of 0.98. The standard deviation of 0.23 indicates some variation in energy levels across tracks.
Loudness: Tracks have an average loudness of -10.51 dB, with the loudest track reaching -0.74 dB and the quietest track being -18.62 dB.
Tempo: The average tempo is 114.7 beats per minute (BPM), with tracks ranging from 61.89 BPM to 195.24 BPM.
Release Year: The tracks span from 1981 to 2023, with a median release year of 2017.5, showing that most tracks are relatively recent.
Explicit Content: Only 2% of the tracks are labeled as having explicit content.





Mahnıların Müddəti: Orta hesabla mahnıların uzunluğu təxminən 4.25 dəqiqədir və ən uzun mahnı təxminən 8 dəqiqədir.
Populyarlıq: Orta populyarlıq göstəricisi 17.6-dır, maksimum dəyər 39, minimum isə 0-dır. Əksər mahnılar aşağı populyarlıq göstəricilərinə sahibdir.
Enerji Səviyyələri: Orta enerji göstəricisi 0.45-dir və ən yüksək dəyər 0.98-dir. 0.23-lük standart sapma enerji səviyyələrinin mahnılar arasında dəyişkən olduğunu göstərir.
Səslilik: Mahnıların orta səsliliyi -10.51 dB-dir, ən yüksək səslilik -0.74 dB, ən aşağı isə -18.62 dB-dir.
Tempolar: Orta tempo 114.7 döyünmə dəqiqədir (BPM), mahnılar isə 61.89 BPM-dən 195.24 BPM-ə qədər dəyişir.
Buraxılış İli: Mahnılar 1981-ci ildən 2023-cü ilə qədər yayımlanıb, median buraxılış ili isə 2017.5-dir, bu da göstərir ki, əksər mahnılar nisbətən yenidir.
Eksplisit Məzmun: Mahnıların yalnız 2%-i eksplisit məzmuna malikdir.

In [ ]:
music_df.shape

(100, 21)

In [ ]:
music_df.isnull().sum()

,0
Track Name,0
Artists,0
Album Name,0
Album ID,0
Track ID,0
Popularity,0
Release Date,0
Duration (ms),0
Explicit,0
External URLs,0


In [ ]:
music_df.duplicated().sum()


1

In [ ]:
music_df.drop_duplicates(inplace=True)


In [ ]:
#Convert 'Duration (ms)' to 'Duration (minutes)'
music_df['Duration (minutes)'] = music_df['Duration (ms)'] / 60000

In [ ]:
#Extract 'Release Year' from 'Release Date'
music_df['Release Year'] = pd.to_datetime(music_df['Release Date'], errors='coerce').dt.year

In [ ]:
# Generate a 'Track Length Category' based on duration
music_df['Track Length Category'] = pd.cut(music_df['Duration (minutes)'],
                                           bins=[0, 2, 4, 6, float('inf')],
                                           labels=['Short', 'Medium', 'Long', 'Very Long'])

In [ ]:
#Create a feature 'Energy Level' based on energy score
music_df['Energy Level'] = pd.cut(music_df['Energy'],
                                  bins=[0, 0.3, 0.6, 1],
                                  labels=['Low', 'Medium', 'High'])


0-2:Short
2-4:Medium
4-6:Long
>6:Very Long

In [ ]:

# music_df['Energy'].unique()

array([0.259 , 0.0461, 0.22  , 0.302 , 0.183 , 0.43  , 0.433 , 0.495 ,
       0.506 , 0.344 , 0.721 , 0.089 , 0.226 , 0.622 , 0.409 , 0.623 ,
       0.296 , 0.51  , 0.414 , 0.505 , 0.599 , 0.557 , 0.27  , 0.189 ,
       0.134 , 0.413 , 0.722 , 0.386 , 0.474 , 0.397 , 0.91  , 0.291 ,
       0.683 , 0.676 , 0.681 , 0.566 , 0.79  , 0.0562, 0.3   , 0.62  ,
       0.492 , 0.152 , 0.181 , 0.266 , 0.286 , 0.139 , 0.367 , 0.573 ,
       0.545 , 0.408 , 0.148 , 0.243 , 0.797 , 0.295 , 0.0542, 0.238 ,
       0.601 , 0.693 , 0.431 , 0.648 , 0.13  , 0.652 , 0.951 , 0.267 ,
       0.17  , 0.304 , 0.272 , 0.16  , 0.206 , 0.323 , 0.415 , 0.459 ,
       0.735 , 0.673 , 0.866 , 0.911 , 0.755 , 0.615 , 0.362 , 0.365 ,
       0.377 , 0.824 , 0.803 , 0.559 , 0.981 , 0.199 , 0.329 , 0.777 ,
       0.854 , 0.42  , 0.214 , 0.741 , 0.83  , 0.273 , 0.312 ])

In [ ]:
# music_df['Duration (minutes)'].unique()

array([3.11845   , 3.03411667, 2.98045   , 5.37386667, 3.33758333,
       5.956     , 5.37488333, 6.164     , 4.1823    , 5.76566667,
       4.11535   , 4.1807    , 5.55883333, 3.93443333, 3.99615   ,
       4.84498333, 4.97501667, 3.81888333, 5.41155   , 3.39778333,
       4.16678333, 3.20758333, 5.06466667, 4.58941667, 6.8417    ,
       7.99893333, 4.11186667, 5.90443333, 3.2268    , 5.846     ,
       3.17555   , 5.05583333, 2.36233333, 5.87123333, 3.95378333,
       4.01081667, 4.4801    , 4.11231667, 2.92733333, 4.34038333,
       3.73711667, 1.60826667, 2.97143333, 2.27511667, 6.77225   ,
       4.67555   , 5.01621667, 3.95195   , 6.769     , 3.25398333,
       4.96463333, 4.1655    , 2.98845   , 6.28898333, 3.92271667,
       5.09666667, 4.78243333, 2.108     , 3.51146667, 4.86356667,
       4.83      , 4.35693333, 2.50166667, 2.05568333, 4.34873333,
       7.10063333, 3.42266667, 3.79801667, 3.75858333, 4.73386667,
       3.11988333, 4.97591667, 3.03333333, 4.32108333, 2.12158

In [ ]:
music_df.to_csv('music_data.csv', index=False)


In [ ]:
from google.colab import files
files.download('music_data.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>